In [2]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib as mpl
from typing import *

Compute the vamp_eq(2) = 1+(EV2)^2 in summary for optimisation.

! vampeq2 only contains values for the 2nd process.

In [13]:
protein='BBA'
batch_num = 1
raw_path = f'../{protein}/raw_batch{batch_num}.h5'
summary_path = f'../{protein}/summary_batch{batch_num}.h5'
timescales = pd.DataFrame(pd.read_hdf(raw_path, key='timescales'))

timescales.reset_index(inplace=True)
timescales.rename(mapper={0: 'value'}, axis=1, inplace=True)

evs = timescales.copy(deep=True)
vampeq2 = timescales.query('process==2').copy(deep=True)

In [14]:
evs['evs'] = np.exp(np.divide(-evs['lag'], evs['value']))

In [10]:
evs_summary = evs.groupby(['hp_ix', 'lag', 'process'])['evs'].agg(
            median=lambda x: np.nanquantile(x, 0.5),
            lb=lambda x: np.nanquantile(x, 0.025),
            ub=lambda x: np.nanquantile(x, 0.975),
            count=lambda x: x.shape[0]-x.isna().sum()
)
evs_summary.reset_index(inplace=True)
evs_summary.head()

In [12]:
evs_summary.to_hdf(summary_path, key='evs')

---

In [17]:
vampeq2['vamp2eq'] = np.exp(2*np.divide(-vampeq2['lag'], vampeq2['value']))+1

In [21]:
vampeq2_summary = vampeq2.groupby(['hp_ix', 'lag', 'process'])['vamp2eq'].agg(
            median=lambda x: np.nanquantile(x, 0.5),
            lb=lambda x: np.nanquantile(x, 0.025),
            ub=lambda x: np.nanquantile(x, 0.975),
            count=lambda x: x.shape[0]-x.isna().sum()
)
vampeq2_summary.reset_index(inplace=True)

In [22]:
vampeq2_summary.to_hdf(summary_path, key='vampeq2')